In [1]:
import itk
from itkwidgets import view

In [3]:
img = itk.imread('./data/Task09_Spleen/imagesTr/spleen_10.nii.gz')
print(img)

Image (000001FEEFB53070)
  RTTI typeinfo:   class itk::Image<float,3>
  Reference Count: 1
  Modified Time: 795
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 609
  UpdateMTime: 794
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  Spacing: [0.976562, 0.976562, 5]
  Origin: [499.023, 499.023, 0]
  Direction: 
-1 0 0
0 -1 0
0 0 1

  IndexToPointMatrix: 
-0.976562 0 0
0 -0.976562 0
0 0 5

  PointToIndexMatrix: 
-1.024 0 0
0 -1.024 0
0 0 0.2

  Inverse Direction: 
-1 0 0
0 -1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (000001FEEFFD4260)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int64,float>

In [13]:
minmax = itk.MinimumMaximumImageCalculator[itk.Image[itk.F, 3]].New(Image=img)
minmax.Compute()
print(minmax.GetMinimum(), minmax.GetMaximum())

itk.imwrite(img, './tmp.dcm')

-1024.0 3072.0


RuntimeError: C:\P\IPP\ITK-source\ITK\Modules\IO\GDCM\src\itkGDCMImageIO.cxx:1204:
itk::ERROR: itk::ERROR: GDCMImageIO(000001FEF47D5600): A Floating point buffer was passed but the stored pixel type was not specified.This is currently not supported

In [30]:
import numpy as np
def float_image_to_dicom(img, fp):
    gdcm2 = itk.GDCMImageIO.New()

    rescaler = itk.RescaleIntensityImageFilter[itk.Image[itk.F,3], itk.Image[itk.F,3]].New(Input=img, OutputMinimum=0, OutputMaximum=itk.NumericTraits.US.max())
    caster = itk.CastImageFilter[itk.Image[itk.F,3], itk.Image[itk.US, 3]].New(Input=rescaler.GetOutput())
    caster.Update()
    img_intercept = -rescaler.GetShift()
    img_scale = rescaler.GetScale()
    print('intercept(shift): {}, scale: {}'.format(img_intercept, img_scale))

    npimg = itk.array_from_image(img)
    print('img min: {}, img max: {}'.format(np.min(npimg), np.max(npimg)))
    intercept_tag = '0028|1052'
    scale_tag = '0028|1053'

    # consider also bits allocated, bits stored, pixel representation, sample per pixel
    meta = gdcm2.GetMetaDataDictionary()
    meta.Set(intercept_tag, itk.MetaDataObject[str].New(MetaDataObjectValue=str(img_intercept)))
    meta.Set(scale_tag, itk.MetaDataObject[str].New(MetaDataObjectValue=str(img_scale)))

    writer = itk.ImageSeriesWriter[itk.Image[itk.US,3],itk.Image[itk.US,3]].New(ImageIO=gdcm2, FileName=fp, Input=caster)
    writer.Update()
    
float_image_to_dicom(img, './tmp.dcm')

intercept(shift): -16383.75, scale: 15.999755859375
img min: -1024.0, img max: 3072.0


In [34]:
img2 = itk.imread('./tmp.dcm')
#img2 = itk.imread('./tmp.dcm', pixel_type=itk.US)
print(img2)

minmax = itk.MinimumMaximumImageCalculator[itk.Image[itk.D, 3]].New(Image=img2)
minmax.Compute()
print('img2 min: {}, max: {}'.format(minmax.GetMinimum(), minmax.GetMaximum()))

Image (000001FEF4E6A120)
  RTTI typeinfo:   class itk::Image<double,3>
  Reference Count: 1
  Modified Time: 4523
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 4343
  UpdateMTime: 4522
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  Spacing: [0.976562, 0.976562, 5]
  Origin: [499.023, 499.023, 0]
  Direction: 
-1 0 0
0 -1 0
0 0 1

  IndexToPointMatrix: 
-0.976562 0 0
0 -0.976562 0
0 0 5

  PointToIndexMatrix: 
-1.024 0 0
0 -1.024 0
0 0 0.2

  Inverse Direction: 
-1 0 0
0 -1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (000001FEFA9BD5F0)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int64,do

In [35]:
dict(img2)

{'0008|0016': '1.2.840.10008.5.1.4.1.1.7.3',
 '0008|0018': '1.2.826.0.1.3680043.2.1125.1.44133476712029325138405945436658329',
 '0008|0020': '20210217',
 '0008|0030': '144949.334603 ',
 '0008|0050': '',
 '0008|0060': 'OT',
 '0008|0090': '',
 '0010|0010': '',
 '0010|0020': '',
 '0010|0030': '',
 '0010|0040': '',
 '0020|000d': '1.2.826.0.1.3680043.2.1125.1.16850624303170220119321237213959627',
 '0020|000e': '1.2.826.0.1.3680043.2.1125.1.65807568054882542831771648579086647',
 '0020|0010': '',
 '0020|0011': '',
 '0020|0013': '',
 '0020|0052': '1.2.826.0.1.3680043.2.1125.1.35654883809507999255362117304536284',
 '0028|0002': '1',
 '0028|0004': 'MONOCHROME2 ',
 '0028|0008': '55',
 '0028|0009': '(5200,9230)',
 '0028|0010': '512',
 '0028|0011': '512',
 '0028|0100': '16',
 '0028|0101': '16',
 '0028|0102': '15',
 '0028|0103': '0',
 '0028|1052': '-16383.75 ',
 '0028|1053': '15.999755859375 ',
 '0028|1054': 'US',
 'origin': array([  0.        , 499.02319336, 499.02319336]),
 'spacing': array([5.   

In [1]:

dummy_names = ['Joe', 'Johnny', 'Jack', 'Jen', 'Jill']
dicomIO = itk.GDCMImageIO.New()
dicomIO = itk.GDCMImageIO.New()
reader = itk.ImageSeriesReader[itk.Image[itk.F,3]].New(FileName='./data/case1_068.dcm')
reader.SetImageIO(dicomIO)
reader.ForceOrthogonalDirectionOff()
reader.Update()


NameError: name 'itk' is not defined

In [33]:
import pydicom
#! pip install pydicom
pydicom.read_file('./data/case1_068.dcm')

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 198
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.46.670589.11.0.0.11.4.2.0.5526.5.2220.2004042613240932485
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.50.20060201.4.1
(0002, 0013) Implementation Version Name         SH: 'JIVEX_TK_41'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M_FFE', 'M', 'FFE']
(0008, 0012) Instance Creation Date              DA: '20040426'
(0008, 0013) Instance Creation Time              TM: '132601'
(0008, 0014) Instance Creator UID                UI: 1.3.46.670589.11.5526.5
(

In [34]:
img3 = reader.GetOutput()
print(reader.GetMetaDataDictionaryArray())
print(img3)
print(img3.GetMetaDataDictionary())

<itk.ITKCommonBasePython.vectoritkMetaDataDictionary; proxy of <Swig Object of type 'std::vector< itkMetaDataDictionary * > *' at 0x0000026C0A8BDEA0> >
Image (0000026C069C5D20)
  RTTI typeinfo:   class itk::Image<float,3>
  Reference Count: 3
  Modified Time: 1071
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0000026BFEDD0830) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 984
  UpdateMTime: 1072
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 1]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 1]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 1]
  Spacing: [0.732422, 0.732422, 1]
  Origin: [-177.711, -57.827, 185.987]
  Direction: 
1 0 0
0 0.71103 0.703161
0 -0.703161 0.71103

  IndexToPointMatrix: 
0.732422 0 0
0 0.520774 0.703161
0 -0.515011 0.71103

  PointToIndexMatrix: 

In [39]:
arr = reader.GetMetaDataDictionaryArray()
m = arr[0]

In [51]:
t = m[m.GetKeys()[0]]
t

'ISO_IR 100'

In [53]:
for k in m.GetKeys():
    print(k, m[k])

0008|0005 ISO_IR 100
0008|0008 ORIGINAL\PRIMARY\M_FFE\M\FFE
0008|0012 20040426
0008|0013 132601
0008|0014 1.3.46.670589.11.5526.5
0008|0016 1.2.840.10008.5.1.4.1.1.4
0008|0018 1.2.276.0.50.192168001099.7810872.14547392.527
0008|0020 20010101
0008|0021 20070511
0008|0023 20070511
0008|0030 113402
0008|0031 113402
0008|0033 113402
0008|0050 11788759296818
0008|0060 MR
0008|0070 Philips Medical Systems 
0008|0080 Anonymized Hospital 
0008|0081 Anonymized Address
0008|0090 Dr. Anonymous 
0008|1010 Any Station 
0008|1030 MRT Sakroiliakalgelenke 
0008|103e Rheuma     *ISG/THRI/KM 
0008|1040 
0008|1080 
0008|1090 Intera
0010|0010 Fall 1
0010|0020 11788759296811
0010|0030 19000101
0010|0032 0000
0010|0040 O 
0010|1030 88.0
0010|2000 
0010|2110 
0010|2160 
0010|2180 
0010|21b0 
0010|21c0 4
0010|4000 
0018|0020 GR
0018|0021 SP
0018|0022 FS
0018|0023 
0018|0024 
0018|0050 2.0 
0018|0080 7.73110008239746
0018|0081 3.70510005950927
0018|0082 0.0 
0018|0083 3.0 
0018|0084 63.8923629999999
0018|0085 

RuntimeError: Can't downcast to a specialization of MetaDataObject

In [54]:
%debug

> c:\src\venv-3.7.6-prebuilt\lib\site-packages\itkextras.py(1200)down_cast()
   1198         raise RuntimeError(
   1199             "Can't downcast to a specialization of %s" %
-> 1200             className)
   1201     else:
   1202         return t.cast(obj)



ipdb>  k


*** NameError: name 'k' is not defined


ipdb>  u


> c:\src\venv-3.7.6-prebuilt\lib\site-packages\itk\itkcommonbasepython.py(895)__getitem__()
    893         import itk
    894         obj = self.Get(key)
--> 895         return itk.down_cast(obj).GetMetaDataObjectValue()
    896     def __len__(self):
    897         return self.GetKeys().size()



ipdb>  key


'ITK_original_direction'


ipdb>  q


In [11]:
view(img2)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [55]:
view(img)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [92]:
import numpy as np

src_dir = './data/Task09_Spleen/imagesTr'
src_files = ['spleen_10.nii.gz', 'spleen_12.nii.gz', 'spleen_13.nii.gz', 'spleen_14.nii.gz', 'spleen_16.nii.gz']
label_dir = './data/Task09_Spleen/labelsTr'

from pathlib import Path
import os
outdir = './outdir'
os.makedirs(outdir, exist_ok=True)
for f in src_files:
    base = f.replace('.nii.gz', '')
    dicom_f = base + '.dcm'
    label_f = base + '.mha'
    img = itk.imread(str(Path(src_dir + '/' + f)))
    label_img = itk.imread(str(Path(label_dir + '/' + f)))
    
    float_image_to_dicom(img, str(Path(outdir + '/' + dicom_f)))
    itk.imwrite(label_img, str(Path(outdir + '/' + label_f)))


intercept(shift): -16383.75, scale: 15.999755859375
img min: {}, img max: {} -1024.0 3072.0
intercept(shift): -16383.75, scale: 15.999755859375
img min: {}, img max: {} -1024.0 3072.0
intercept(shift): -30324.374152733846, scale: 29.613646633529147
img min: {}, img max: {} -1024.0 1189.0
intercept(shift): -23382.522648083625, scale: 22.834494773519165
img min: {}, img max: {} -1024.0 1846.0
intercept(shift): -25419.636363636364, scale: 24.823863636363637
img min: {}, img max: {} -1024.0 1616.0


In [86]:
%debug

> c:\src\venv-3.7.6-prebuilt\lib\site-packages\itktemplate.py(546)_NewImageReader()
    544             imageIO = kwargs["ImageIO"]
    545         else:
--> 546             imageIO = itk.ImageIOFactory.CreateImageIO(inputFileName, itk.CommonEnums.IOFileMode_ReadMode)
    547         if not imageIO:
    548             msg = ""



ipdb>  inputFileName


WindowsPath('C:/src/ParaViewMedical-MONAI/data/Task09_Spleen/imagesTrspleen_10.nii.gz')


ipdb>  q


In [80]:
img4 = itk.imread('out.dcm')
print(img)
print(img4)

Image (0000026BAB1AEA70)
  RTTI typeinfo:   class itk::Image<float,3>
  Reference Count: 5
  Modified Time: 418
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 232
  UpdateMTime: 417
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  Spacing: [0.976562, 0.976562, 5]
  Origin: [499.023, 499.023, 0]
  Direction: 
-1 0 0
0 -1 0
0 0 1

  IndexToPointMatrix: 
-0.976562 0 0
0 -0.976562 0
0 0 5

  PointToIndexMatrix: 
-1.024 0 0
0 -1.024 0
0 0 0.2

  Inverse Direction: 
-1 0 0
0 -1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0000026BAB89A520)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int64,float>

In [64]:
itk.ImageSeriesWriter.GetTypes()

<itkTemplate itk::ImageSeriesWriter>
Options:
  [<class 'itk.itkImagePython.itkImageCF2'>, <class 'itk.itkImagePython.itkImageCF2'>]
  [<class 'itk.itkImagePython.itkImageCF3'>, <class 'itk.itkImagePython.itkImageCF2'>]
  [<class 'itk.itkImagePython.itkImageCF3'>, <class 'itk.itkImagePython.itkImageCF3'>]
  [<class 'itk.itkImagePython.itkImageCVF22'>, <class 'itk.itkImagePython.itkImageCVF22'>]
  [<class 'itk.itkImagePython.itkImageCVF23'>, <class 'itk.itkImagePython.itkImageCVF23'>]
  [<class 'itk.itkImagePython.itkImageCVF32'>, <class 'itk.itkImagePython.itkImageCVF32'>]
  [<class 'itk.itkImagePython.itkImageCVF33'>, <class 'itk.itkImagePython.itkImageCVF33'>]
  [<class 'itk.itkImagePython.itkImageCVF42'>, <class 'itk.itkImagePython.itkImageCVF42'>]
  [<class 'itk.itkImagePython.itkImageCVF43'>, <class 'itk.itkImagePython.itkImageCVF43'>]
  [<class 'itk.itkImagePython.itkImageD2'>, <class 'itk.itkImagePython.itkImageD2'>]
  [<class 'itk.itkImagePython.itkImageD3'>, <class 'itk.itkIma